In [28]:
import os
transcript_dir = r"..\transcripts"

transcripts = {}
for filename in os.listdir(transcript_dir):
    with open(os.path.join(transcript_dir, filename), 'r') as f:
        transcripts[filename.split(".")[0]] = f.read()

transcripts.keys()

dict_keys(['4tools', 'baselifesciences', 'bookstar', 'calmsea', 'deel', 'explorer', 'keeper', 'lasebane', 'mode', 'ss', 'stackcentral', 'xp'])

In [2]:
labels = {
    "4tools": """Priority: “Understand content workflow with multiple contractors so you can increase your engagement size from 10k to 20k" (see 3:07 and 8:41)
Event: “Trial expires in 8 days” (see 3:07 and 8:41)
Date: <2 weeks from now> (see 43:18)
Message: Hi Ryan, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial has ended. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "bookstar": """Priority: “Traditional outreach is failing and needs to invest in social selling but marketing is stretched thin" (see 2:40)
Event: “Kate to talk to her boss” (see 31:24)
Date: <next June> (see 38:26)
Message: Hi Kate, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial expires tomorrow. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "xp": """Priority: “Get rid of Terraform Cloud in Q3 because support is not great" (see 3:42)
Event: “TCF contracts expires” (see 31:24)
Date: <next December> (see 5:03)
Message: Hi Mike, last we spoke, it was important to move off Terraform Cloud in Q3. I know your contract ends in December. Wanted to reconnect to see if we’re on track to replace TFC.""",
    
    "mode": """Priority: “Increase thought leadership as part of ABM execution" (see 22:42)
Event: “ABM research is complete” (see 9:53)
Date: <next December> (see 5:03)
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.""",

    }

In [3]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [4]:
import os
import sys

os.environ['DSP_CACHEBOOL'] = 'false'
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(os.getcwd(), 'cache')

import dspy

temperature = 0.0

lm = dspy.LM(
    model="azure/gpt-4o",
    temperature=temperature,
    api_base=os.environ["AZURE_API_BASE"],
    api_key=os.environ["AZURE_API_KEY"],
    api_version=os.environ["AZURE_API_VERSION"],
    # cache=False
)


dspy.configure(lm=lm)

k:\kj\Projects\sales_actions\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


k:\kj\Projects\sales_actions\notebooks\cache\compiler


## No specific prompt

In [6]:
action_item = dspy.Predict('sales_call_transcript -> hidden_action_item',)

response = action_item(sales_call_transcript = transcripts['xp'])
print(response.hidden_action_item)

Mike Liv needs to sign up for an account and set up the necessary cloud and VCS integrations before the POC kickoff call next week. Additionally, he should check with his procurement team about the NDA requirements and share the necessary documents in the Slack channel.


## Custom prompt signature

In [5]:
class SalesActionItem(dspy.Signature):
    """Find hidden action items in the given sales call transcript. Identify things that sales rep has to do in future. Only provide the action items for tasks that are to be done atleast 2 weeks from the day of the call."""

    sales_call_transcript = dspy.InputField(desc="Sales call transcript with a potential customer.")
    hidden_action_item = dspy.OutputField()



action_item = dspy.Predict(SalesActionItem, n=1)

In [8]:
response = action_item(sales_call_transcript = transcripts['4tools'], config=dict(temperature=0.0))
print(response.hidden_action_item)

Homer needs to send Taylor the WordPress setup instructions and extend her account trial by another week.


## dataset, metric, tuning

In [ ]:
labeled_dataset = []

for k,v in labels.items():
    labeled_dataset.append(dspy.Example(sales_call_transcript = transcripts[k], hidden_action_item = v).with_inputs("sales_call_transcript"))

# labeled_dataset

In [10]:
class Assess(dspy.Signature):
    """Evaluate the quality of a system's answer to a question according to a given criterion."""
    
    criterion: str = dspy.InputField(desc="The evaluation criterion.")
    sales_call_transcript: str = dspy.InputField(desc="The sales call transcript that system had to analyze.")
    ground_truth_answer: str = dspy.InputField(desc="An expert written Ground Truth Answer for the sales call transcript.")
    predicted_answer: str = dspy.InputField(desc="The system generated action item for given transcript.")
    rating: float = dspy.OutputField(desc="A float rating between 1 and 5")


def metric(gt, pred, trace=None):
        alignment_criterion = "How accurately does system predict the most important action items in the sales call transcript?"
        return dspy.TypedPredictor(Assess)(criterion=alignment_criterion,
            sales_call_transcript=gt.sales_call_transcript,
            ground_truth_answer=gt.hidden_action_item,
            predicted_answer=pred.hidden_action_item
        ).rating

In [11]:
from dspy.teleprompt import BootstrapFewShot

for i in range(1, 4, 1):
    teleprompter = BootstrapFewShot(metric=metric, max_bootstrapped_demos=i, max_rounds=1)
    compiled = teleprompter.compile(action_item, trainset=labeled_dataset)
    # compiled_score = evaluate(compiled, metric=MetricWrapper)
    # print(f"\n\033[91mCompiled Score at Demos = {i}: {compiled_RAG_score}\n")


 25%|██▌       | 1/4 [00:06<00:20,  6.84s/it]


Bootstrapped 1 full traces after 2 examples in round 0.



 50%|█████     | 2/4 [00:07<00:07,  3.88s/it]


Bootstrapped 2 full traces after 3 examples in round 0.



 75%|███████▌  | 3/4 [00:09<00:03,  3.23s/it]


Bootstrapped 3 full traces after 4 examples in round 0.


In [12]:
# compiled.save(r'dspy_files\bsfs_1.json')

In [8]:
# compiled = action_item.deepcopy()
# compiled.load(path=r'dspy_files\bsfs.json')

## Initial test results

In [16]:
for k,v in transcripts.items():
    res = compiled(sales_call_transcript = v)
    print(f"{k}: ")
    print(res.hidden_action_item)
    print("\n-----------------------------------\n")

4tools: 
Priority: “Test the workflow into assigning it to writers, getting pieces written, and pushing it directly into our sandbox site" (see 41:04)
Event: “Taylor's trial extension” (see 41:53)
Date: <next two weeks> (see 43:18)
Message: Hi Marc, last we spoke, it was important to test the workflow into assigning it to writers, getting pieces written, and pushing it directly into your sandbox site. I have extended Taylor's trial for another week. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.

-----------------------------------

baselifesciences: 
Priority: “Schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into" (see 8:39)
Event: “Team's availability in September or October” (see 8:39)
Date: <September or October> (see 8:39)
Message: Hi Jonathan, last we spoke, it was important to schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into. Wanted 

In [13]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset=labeled_dataset, num_threads=1, display_progress=True, display_table=False)


compiled_score = evaluate(compiled, metric=metric)
print(f"Compiled score: {compiled_score}")

Average Metric: 16.0 / 4  (400.0): 100%|██████████| 4/4 [00:48<00:00, 12.12s/it]
Compiled score: 400.0


## Different optimization metric

In [14]:
for k,v in transcripts.items():
    res = compiled(sales_call_transcript = v)
    print(f"{k}: ")
    print(res.hidden_action_item)
    print("\n-----------------------------------\n")

4tools: 
Priority: “Test the workflow into assigning it to writers, getting pieces written, and pushing it directly into our sandbox site" (see 41:04)
Event: “Taylor's trial extension” (see 41:53)
Date: <next two weeks> (see 43:18)
Message: Hi Marc, last we spoke, it was important to test the workflow into assigning it to writers, getting pieces written, and pushing it directly into your sandbox site. I have extended Taylor's trial for another week. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.

-----------------------------------

baselifesciences: 
Priority: “Schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into" (see 8:39)
Event: “Team's availability for a workshop” (see 9:26)
Date: <September or October> (see 9:26)
Message: Hi Jonathan, last we spoke, it was important to schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into. Wanted to reconn

## MIPRO

In [47]:


class GenerateInstruction(dspy.Signature):
    """You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (sales_call_transcript and hidden_action_item) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative."""

    basic_instruction = dspy.InputField(desc="The initial instructions before optimization")
    proposed_instruction = dspy.OutputField(desc="The improved instructions for the language model")
    proposed_prefix_for_output_field = dspy.OutputField(
        desc="The string at the end of the prompt, which will help the model start solving the task",
    )

## this is not working right now
# def mipro_metric(gt, pred, trace=None):
#     alignment_criterion = "How accurately does system predict the most important action items in the sales call transcript?"
#     return dspy.TypedPredictor(GenerateInstruction)(criterion=alignment_criterion,
#         basic_instruction=gt.sales_call_transcript,
#         proposed_instruction=gt.hidden_action_item,
#         predicted_answer=pred.hidden_action_item
#     )

In [ ]:
from dspy.teleprompt import MIPROv2

teleprompter = MIPROv2(
    metric=metric,
    num_candidates=7,
    init_temperature=0.5,
    verbose=True,
    num_threads=4,
    task_model=lm,
    prompt_model=lm,
)

print(f"Optimizing program with MIPRO...")
optimized_program = teleprompter.compile(
    action_item.deepcopy(),
    trainset=labeled_dataset,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    num_trials=5,
    minibatch_size=2,
    minibatch_full_eval_steps=2,
    minibatch=True, 
    requires_permission_to_run=False,
)

In [21]:
# optimized_program.save(r'dspy_files\mipro.json')

### MIPRO result first iteration

In [27]:
for k,v in transcripts.items():
    res = optimized_program(sales_call_transcript = v)
    print(f"{k}: ")
    print(res.hidden_action_item)
    print("\n-----------------------------------\n")

4tools: 
Priority: “Test the workflow into assigning it to writers, getting pieces written, and pushing it directly into our sandbox site" (see 41:04)
Event: “Taylor's trial extension” (see 41:53)
Date: <next two weeks> (see 43:18)
Message: Hi Marc, last we spoke, it was important to test the workflow into assigning it to writers, getting pieces written, and pushing it directly into your sandbox site. I have extended Taylor's trial for another week. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.

-----------------------------------

baselifesciences: 
Priority: “Schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into" (see 8:39)
Event: “Team's availability in September or October” (see 8:39)
Date: <September or October> (see 8:39)
Message: Hi Jonathan, last we spoke, it was important to schedule a workshop to focus on a feature that the team might want to use but haven't had time to dive into. Wanted 